In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
import re
from time import sleep

import warnings
warnings.filterwarnings('ignore')

In [6]:
gecko_driver_path = "geckodriver.exe"
firefox_options = webdriver.FirefoxOptions()
firefox_service = FirefoxService(executable_path=gecko_driver_path)
driver = webdriver.Firefox(service=firefox_service, options=firefox_options)
url = "https://www.mystore411.com/store/sl/A"
driver.get(url)

In [7]:
first_page_store_list = driver.find_element(By.ID,"sl_popular")
li_elements = first_page_store_list.find_elements(By.TAG_NAME,"li")
li_texts = [li.text for li in li_elements]

In [ ]:
li_texts = ['ALDI']

In [10]:
data = []
for link_text in li_texts:
    target_link = driver.find_element(By.LINK_TEXT, link_text)
    target_link.click()
    second_page_country = driver.find_element(By.ID,"slim_sidebar")
    list_elements = second_page_country.find_elements(By.TAG_NAME,"li")
    country_list = [li.text for li in list_elements]
    germany = country_list.count('Germany')

    if germany:
        print('Country Found')
        target_link = second_page_country.find_element(By.LINK_TEXT, "Germany")
        target_link.click()
        germany = 0
    else:
        driver.back()
        continue

    # try:
    #     table_head = driver.find_element(By.XPATH, "//th[contains(text(), 'Store By State')]")
    # except Exception as e:
    #     print(e)
    #     driver.back()
    #     continue

    list_elements = driver.find_elements(By.TAG_NAME,"td")
    state_list = [td.text for td in list_elements]
    pages = state_list[0]
    states = state_list[1:]
    if len(states) == 0:
        continue
    for state in states:
        target_link = driver.find_element(By.LINK_TEXT, state)
        target_link.click()
        list_elements = driver.find_elements(By.XPATH, '//*[@id="main"]/table/tbody/tr/td/a')#driver.find_elements(By.TAG_NAME,"td")
        stores_list = [td.text for td in list_elements]
        for store in stores_list:
            target_link = driver.find_element(By.LINK_TEXT, store)
            target_link.click()
            store_name = driver.find_element(By.CLASS_NAME, 'store_title').text
            address = driver.find_element(By.CLASS_NAME, 'adr').text
            list_elements = driver.find_elements(By.TAG_NAME, 'p')

            hours = [td.text for td in list_elements]

            # Join the list into a single string for easier matching
            info_text = '\n'.join(hours)

            # Use regex to extract the schedule information
            schedule_pattern = re.compile(r'Mo:.*?So:.*?(?=\n\n)', re.DOTALL)
            schedule_match = schedule_pattern.search(info_text)
            schedule_info = schedule_match.group()
            print(store_name)
            data.append([store_name,address,schedule_info])
            driver.back()
        driver.back()
    driver.back()

Country Found


NoSuchElementException: Message: Unable to locate element: APO; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16


In [14]:
driver.quit()

In [58]:
import pandas as pd
df = pd.DataFrame(data,columns=['name','address','timing'])

In [59]:
df.name = df.name.str.replace('- Location & Hours','').str.strip()
df['type'] = df['name'].str.extract(r'\((.*?)\)')
df['name'] = df['name'].str.replace(r'\([^)]*\)', '')

In [60]:
df[['street_locality','region_postcode']] = df.address.str.split(',', 1, expand=True)
df[['street','locality']] = df.street_locality.str.split('\n', 1, expand=True)
df[['region','postcode']] = df.address.str.split(' ', 1, expand=True)

In [61]:
df.drop(['street_locality','region_postcode'],axis=1,inplace=True)

In [63]:
df.sample()

,name,address,timing,type,street,locality,region,postcode
14,ALDI Nord - Embser Landstraße 4 - 6 in Achim,"Embser Landstraße 4 - 6\nAchim, Achim 28832",Mo: 07:00-21:00\nDi: 07:00-21:00\nMi: 07:00-21...,Grocery Supermarket,Embser Landstraße 4 - 6,Achim,Embser,"Landstraße 4 - 6\nAchim, Achim 28832"


In [64]:
df.to_csv('./demo.csv',index=False)